In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from secret import username, password

In [2]:
#importing videogame database
file = os.path.join("Resources", "vgsales.csv")
vgdf = pd.read_csv(file)
vgdf

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


### Cleaning

In [3]:
#removing older platforms of which we can't gather reviews online per Tanner
to_drop = ['NES', 'SNES', 'SAT', '2600', 'GB', 'GEN', 'NG', 'SCD', 'WS', '3DO', 'TG16', 'GG', 'PCFX']
vgdf_plat_drop = vgdf[~vgdf['Platform'].isin(to_drop)]

#removing rows with null values
vgdf_platrow_clean = vgdf_plat_drop.dropna()

#removing duplicate rows with same game name/platform
vgdf_final = vgdf_platrow_clean.drop_duplicates(subset=['Name', 'Platform'], keep='first')
vgdf_final

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


#### Publisher ID Table

In [4]:
#creating publisher table with id determined by its index #
vg_publisher_df = pd.DataFrame({'publisher':vgdf_final['Publisher'].unique()})
vg_publisher_df.index.name='publisher_id'
vg_publisher_df

,publisher
publisher_id,
0,Nintendo
1,Microsoft Game Studios
2,Take-Two Interactive
3,Sony Computer Entertainment
4,Activision
...,...
519,Inti Creates
520,Takuyo
521,Interchannel-Holon


#### Platform ID Table

In [5]:
#creating platform table with id determined by its index #
vg_platform_df = pd.DataFrame({'platform_name':vgdf_final['Platform'].unique()})
vg_platform_df.index.name='id'
vg_platform_df

,platform_name
id,
0,Wii
1,DS
2,X360
3,PS3
4,PS2
5,GBA
6,3DS
7,PS4
8,N64


#### Genre ID Table

In [6]:
vg_genre_df = pd.DataFrame({'genre':vgdf_final['Genre'].unique()})
vg_genre_df.index.name='id'
vg_genre_df

,genre
id,
0,Sports
1,Racing
2,Platform
3,Misc
4,Simulation
5,Action
6,Role-Playing
7,Puzzle
8,Shooter


#### Game Title Table

In [7]:
#transform original dataframe to establish unique game title id's and trim columns bound for their own respective tables
title_df = vgdf_final[["Name", "Year", "Publisher", "Genre"]]
title_df = title_df.rename(columns={"Name": "game_title", "Publisher": "publisher", "Year":"year_released",\
                                    'Genre':'genre'})
title_df

,game_title,year_released,publisher,genre
0,Wii Sports,2006.0,Nintendo,Sports
2,Mario Kart Wii,2008.0,Nintendo,Racing
3,Wii Sports Resort,2009.0,Nintendo,Sports
6,New Super Mario Bros.,2006.0,Nintendo,Platform
7,Wii Play,2006.0,Nintendo,Misc
...,...,...,...,...
16593,Woody Woodpecker in Crazy Castle 5,2002.0,Kemco,Platform
16594,Men in Black II: Alien Escape,2003.0,Infogrames,Shooter
16595,SCORE International Baja 1000: The Official Game,2008.0,Activision,Racing
16596,Know How 2,2010.0,7G//AMES,Puzzle


In [8]:
#drop duplicate titles tied to multiple platforms and establish unique id # for each game title
title_df = title_df.drop_duplicates(subset=['game_title'])
title_df = title_df.reset_index(drop=True)
title_df.index.name = "id"
title_df

,game_title,year_released,publisher,genre
id,,,,
0,Wii Sports,2006.0,Nintendo,Sports
1,Mario Kart Wii,2008.0,Nintendo,Racing
2,Wii Sports Resort,2009.0,Nintendo,Sports
3,New Super Mario Bros.,2006.0,Nintendo,Platform
4,Wii Play,2006.0,Nintendo,Misc
...,...,...,...,...
10659,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,dramatic create,Action
10660,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,Falcom Corporation,Role-Playing
10661,Plushees,2008.0,Destineer,Simulation


In [9]:
#create publisher_id and genre_id dicts for replacing in publisher/genre with their id's in those columns
#use vg_publisher_df/vg_genre_df from before

#publisher_id dict
publisherdict = vg_publisher_df.to_dict('index')

for key in publisherdict:
    publisherdict[key] = publisherdict[key]["publisher"]
    
publisherdict_s = {value:key for key, value in publisherdict.items()}

#genre_id dict
genredict = vg_genre_df.to_dict('index')

for key in genredict:
    genredict[key] = genredict[key]["genre"]
    
genredict_s = {value:key for key, value in genredict.items()}

In [10]:
#replace publisher w/ publisher_id and genre_id in game title df
pub_id_replacer = {
    "publisher": publisherdict_s,
    "genre": genredict_s
}
title_df = title_df.replace(pub_id_replacer)
title_df = title_df.rename(columns={'publisher':'publisher_id', 'genre':'genre_id'})
title_df

,game_title,year_released,publisher_id,genre_id
id,,,,
0,Wii Sports,2006.0,0,0
1,Mario Kart Wii,2008.0,0,1
2,Wii Sports Resort,2009.0,0,0
3,New Super Mario Bros.,2006.0,0,2
4,Wii Play,2006.0,0,3
...,...,...,...,...
10659,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,440,5
10660,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,215,6
10661,Plushees,2008.0,135,4


In [11]:
#convert year_released to int
title_df = title_df.astype({'year_released':int})
title_df

,game_title,year_released,publisher_id,genre_id
id,,,,
0,Wii Sports,2006,0,0
1,Mario Kart Wii,2008,0,1
2,Wii Sports Resort,2009,0,0
3,New Super Mario Bros.,2006,0,2
4,Wii Play,2006,0,3
...,...,...,...,...
10659,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016,440,5
10660,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007,215,6
10661,Plushees,2008,135,4


#### Sales Table

In [12]:
#carve original dataframe to extract sales information - name and platform to be replaced by id's
sales_df = vgdf_final[['Name', 'Platform','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
sales_df

,Name,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Wii Sports,Wii,41.49,29.02,3.77,8.46
2,Mario Kart Wii,Wii,15.85,12.88,3.79,3.31
3,Wii Sports Resort,Wii,15.75,11.01,3.28,2.96
6,New Super Mario Bros.,DS,11.38,9.23,6.50,2.90
7,Wii Play,Wii,14.03,9.20,2.93,2.85
...,...,...,...,...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA,0.01,0.00,0.00,0.00
16594,Men in Black II: Alien Escape,GC,0.01,0.00,0.00,0.00
16595,SCORE International Baja 1000: The Official Game,PS2,0.00,0.00,0.00,0.00
16596,Know How 2,DS,0.00,0.01,0.00,0.00


In [13]:
#create dict using zip to map title name with title index, then map onto sales table to replace game name w/ id
title_id_dict = dict(zip(title_df.game_title, title_df.index))
sales_df['Name'] = sales_df['Name'].map(title_id_dict)
sales_df  

<ipython-input-13-d5914e511844>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Name'] = sales_df['Name'].map(title_id_dict)


,Name,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,0,Wii,41.49,29.02,3.77,8.46
2,1,Wii,15.85,12.88,3.79,3.31
3,2,Wii,15.75,11.01,3.28,2.96
6,3,DS,11.38,9.23,6.50,2.90
7,4,Wii,14.03,9.20,2.93,2.85
...,...,...,...,...,...,...
16593,10662,GBA,0.01,0.00,0.00,0.00
16594,2615,GC,0.01,0.00,0.00,0.00
16595,6031,PS2,0.00,0.00,0.00,0.00
16596,10663,DS,0.00,0.01,0.00,0.00


In [14]:
#rename columns 
sales_df = sales_df.rename(columns={"Name": "title_id", "NA_Sales": "na_sales", "EU_Sales":"eu_sales",\
                                    'JP_Sales':'jp_sales', 'Other_Sales':'other_sales', 'Platform':"platform_id"})

In [15]:
sales_df = sales_df.set_index('title_id')
sales_df

,platform_id,na_sales,eu_sales,jp_sales,other_sales
title_id,,,,,
0,Wii,41.49,29.02,3.77,8.46
1,Wii,15.85,12.88,3.79,3.31
2,Wii,15.75,11.01,3.28,2.96
3,DS,11.38,9.23,6.50,2.90
4,Wii,14.03,9.20,2.93,2.85
...,...,...,...,...,...
10662,GBA,0.01,0.00,0.00,0.00
2615,GC,0.01,0.00,0.00,0.00
6031,PS2,0.00,0.00,0.00,0.00


In [16]:
#convert platform name to platform id # referencing platform table above to create dictionary to map replacement values
platformdict = vg_platform_df.to_dict('index')

for key in platformdict:
    platformdict[key] = platformdict[key]["platform_name"]
    
platformdict_s = {value:key for key, value in platformdict.items()}

plat_replacer = {
    "platform_id": platformdict_s
}

sales_df = sales_df.replace(plat_replacer)
sales_df

,platform_id,na_sales,eu_sales,jp_sales,other_sales
title_id,,,,,
0,0,41.49,29.02,3.77,8.46
1,0,15.85,12.88,3.79,3.31
2,0,15.75,11.01,3.28,2.96
3,1,11.38,9.23,6.50,2.90
4,0,14.03,9.20,2.93,2.85
...,...,...,...,...,...
10662,5,0.01,0.00,0.00,0.00
2615,14,0.01,0.00,0.00,0.00
6031,4,0.00,0.00,0.00,0.00


### Juncture Table

In [17]:
#Create juncture table of title_id's and platform_id's as some titles can be on multiple platforms
#replace game and platform name with respective id's with similar dictionary method

#import subset of original df
title_plat_df = vgdf_final[['Name','Platform']]

#create game id dict for replace method
game_dict = title_df[['game_title']].to_dict('index')

for key in game_dict:
    game_dict[key] = game_dict[key]["game_title"]
    
game_dict_s = {value:key for key, value in game_dict.items()}

#referencing platform dict from above to also go in replace method
title_plat_replacer = {
    "Name": game_dict_s,
    "Platform": platformdict_s
}

#replace names with id's and rename columns
title_plat_id = title_plat_df.replace(title_plat_replacer)
title_plat_id = title_plat_id.rename(columns={'Name':'title_id','Platform':'platform_id'})
title_plat_id

,title_id,platform_id
0,0,0
2,1,0
3,2,0
6,3,1
7,4,0
...,...,...
16593,10662,5
16594,2615,14
16595,6031,4
16596,10663,1


#### Review Helper

In [18]:
# created below tables to assist in formatting of final review_df for Tanner
review_helper = vgdf_final[['Name','Platform']]
review_helper["name_id"] = review_helper["Name"]
review_helper["platform_id"] = review_helper["Platform"]
review_helper = review_helper.replace(title_plat_replacer)
review_helper = review_helper.rename(columns={'Name':'name_id','Platform':'platform_id','name_id':'name','platform_id':'platform'})

<ipython-input-18-02a4af438f97>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_helper["name_id"] = review_helper["Name"]
<ipython-input-18-02a4af438f97>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_helper["platform_id"] = review_helper["Platform"]


In [19]:
review_helper

,name_id,platform_id,name,platform
0,0,0,Wii Sports,Wii
2,1,0,Mario Kart Wii,Wii
3,2,0,Wii Sports Resort,Wii
6,3,1,New Super Mario Bros.,DS
7,4,0,Wii Play,Wii
...,...,...,...,...
16593,10662,5,Woody Woodpecker in Crazy Castle 5,GBA
16594,2615,14,Men in Black II: Alien Escape,GC
16595,6031,4,SCORE International Baja 1000: The Official Game,PS2
16596,10663,1,Know How 2,DS


#### Review Data Import

In [20]:
#import and transform review dataframe for import
review_csv = os.path.join("Resources", "review_df.csv")
review_df = pd.read_csv(review_csv)

review_df.drop(columns=["Unnamed: 0", "Name", "Platform"], inplace=True)

review_df.dropna(inplace=True)

review_df.rename(columns={"Name_id" : "title_id", "Platform_id" : "platform_id", "Review Source": "source",\
                 "Review Score" : "score", "Review Body" : "body"}, inplace=True)

review_df

,title_id,platform_id,source,score,body
0,0,0,Official Nintendo Magazine UK,90.0,The ideal way to get into Wii gaming. It's fun...
1,0,0,GameZone,85.0,"Sure, it lacks goals and can be beaten very qu..."
2,0,0,GameDaily,80.0,Wii Sports certainly won't be hailed as the mo...
3,0,0,GamingExcellence,77.0,"Despite these two solo modes, Wii Sports is re..."
4,0,0,My Gamer,74.0,This is one you can feel confident about showi...
...,...,...,...,...,...
35108,10658,1,IGN,45.0,It's insulting that the developer and publishe...
35109,10658,1,GameShark,42.0,"The game claims to score brain development, bu..."
35110,10658,1,GameDaily,40.0,You'll blow through most of Mega Brain Boost's...
35111,10658,1,NintendoWorldReport,20.0,As far as effectiveness in training goes ... I...


#### Postgres Data Load

In [21]:
rds_connection_string = f'{username}:{password}@localhost:5432/Video_Game_Sales_Review'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [22]:
#genre id table load
vg_genre_df.to_sql(name='genre', con=engine, if_exists='append', index=True)

In [23]:
#platform id table load
vg_platform_df.to_sql(name='platform', con=engine, if_exists='append', index=True)

In [24]:
#publisher id table load
vg_publisher_df.to_sql(name='publisher', con=engine, if_exists='append', index=True)

In [25]:
#game title table load
title_df.to_sql(name='title', con=engine, if_exists='append', index=True)

In [26]:
#title/plat id junction table load
title_plat_id.to_sql(name='title_platform', con=engine, if_exists='append', index=False)

In [27]:
#sales table load
sales_df.to_sql(name='sales', con=engine, if_exists='append', index=True)

In [28]:
#game review table load
review_df.to_sql(name='review', con=engine, if_exists='append', index=False)